## Group 12 Coursework 

In [1]:
import os
import datetime
import time
import json
import http.client
import requests
from html.parser import HTMLParser
from html.entities import name2codepoint

# This is a class for parsing the html returned from the web service endpoint
class MyHTMLParser(HTMLParser):
    
    def __init__(self):
        super(MyHTMLParser, self).__init__()
        self.emotion_type = None
        self.emotion_return = False
        
    def handle_starttag(self, tag, attrs):
        if tag=='h3':
            self.emotion_return = True
    
    def handle_data(self, data):
        if self.emotion_return:
            self.emotion_type = data.strip()
        
    def handle_endtag(self, tag):
        if tag=='h3':
            self.emotion_return = False



Below cell will read user input text, and then make a HTTP request using POST method to the webservice endpoint deployed locally (http://127.0.0.1:5000) to classify the emotion of the user input text.

In [6]:
usertext = input("Please input your text:  ")

Please input your text:  Thank you mate! you are great.


In [7]:
data = {'input_text': usertext}
dbipaddr = '127.0.0.1:5000'

response = requests.post('http://' + dbipaddr, data=data)

parser = MyHTMLParser()

if response.status_code==200:       
    parser.feed(response.text)
    if parser.emotion_type is not None:
        print(f'The classified emotion type is: {parser.emotion_type}')
    else:
        print(f'No emotion can be classified!')

The classified emotion type is: gratitude_pride_relief


In [8]:
!git add .

fatal: not a git repository (or any of the parent directories): .git
